# import and install

In [7]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [34]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# key points using MP holistic

In [35]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilites

In [36]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR 2 RGB
    image.flags.writeable = False                 #image is no longer writeable
    results = model.process(image)                 #make prediction
    image.flags.writeable = True                  #image is writeable
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #RGB 2 BGR
    return image,results

In [37]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) #draw pose connections

# 3 extract values

In [38]:
results.pose_landmarks.landmark[0].visibility

0.9996238946914673

In [39]:
def extract_keypoints(results):
    pose =[]
    for res in results.pose_landmarks.landmark:
        if(res.visibility>0.9):
            test = np.array([res.x, res.y,res.z])
            pose.append(test)
    pose = np.array(pose).flatten()
    return pose

NameError: name 'test' is not defined

# setup folder

In [83]:
# path for exproted data
DATA_PATH = os.path.join("MP_DATA")

#Action that we try to detect
actions = np.array(['good'])

#thirty videos worth of data
no_sequences = 30

#videos are goint to be 30 frames in length
sequence_length =30

In [84]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass


# 5 collect keypoint for trainging

In [85]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Loop through actions
    for action in actions:
        #LOOP through sequneces aka videos
        for sequence in range(no_sequences):
            #Loop through video length aka sequnece length
            for frame_num in range(sequence_length):
                
                #Read feed
                ret,frame = cap.read()

                #make detection
                image , results  = mediapipe_detection(frame,holistic)
                draw_landmarks(image,results)
                
                #Apply collection logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, 'Collectiong frames for {} Video Number {}'.format(action,sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 4, cv2.LINE_AA)
                    #show to screen
                    cv2.imshow('OpenCV Feed',image)
                    cv2.waitKey(1000)
                else:
                    cv2.putText(image, 'Collectiong frames for {} Video Number {}'.format(action,sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 4,cv2.LINE_AA)
                    
                    #show to screen
                    cv2.imshow('OpenCV Feed',image)
                  
                
                #new export keypoints
                keypoints = extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH, action, str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                
                #breaking
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [77]:
np.array(keypoints).shape

(39,)

# 6. preprocess data and create labels

In [78]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [79]:
label_map = {label:num for num, label in enumerate(actions)}

In [80]:
label_map

{'good': 0}

In [86]:
sequences , labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [87]:
np.array(sequences).shape
#행렬의 길이가 다르면 warning을 발생시킴.

(30, 30, 39)

In [88]:
X = np.array(sequences)

In [89]:
X.shape

(30, 30, 39)

In [90]:
y= to_categorical(labels).astype(int)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05)

In [92]:
y_test.shape

(2, 1)

# 7 Build and train LSTM Neural Network

In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [94]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [95]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,39)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))

model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [96]:
res=[0.7,0.2,0.1]

In [97]:
actions[np.argmax(res)]

'good'

In [150]:
#왜 이러한 구조로 구성하였나? -> 
#1. 적은 양의 데이터만 사용할 예정이고
#2. 빠르게 학습시킬 수 있다는 장점과
#3. 실시간으로 평가를 빠르게 내려줄 수 있기 때문입니다.

In [98]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
#crossentropy -> 수치로 표시하기에 유리한 방식으로 출력해주기 때문이다.

In [99]:
model.fit(X_train,y_train,epochs=1000,callbacks=[tb_callback])

Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 2/1000
1/1 [==============================] - 1s 789ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 3/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 4/1000
1/1 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 5/1000
1/1 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 6/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 7/1000
1/1 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 8/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 9/1000
1/1 [==============================] - 0s 44ms/step - loss: 0.0000e+

1/1 [==============================] - 0s 93ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 72/1000
1/1 [==============================] - 0s 60ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 73/1000
1/1 [==============================] - 0s 58ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 74/1000
1/1 [==============================] - 0s 68ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 75/1000
1/1 [==============================] - 0s 47ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 76/1000
1/1 [==============================] - 0s 49ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 77/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 78/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 79/1000
1/1 [==============================] - 0s 46ms/step - loss: 0.0000e+00 -

1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 142/1000
1/1 [==============================] - 0s 57ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 143/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 144/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 145/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 146/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 147/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 148/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 149/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.00

1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 212/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 213/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 214/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 215/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 216/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 217/1000
1/1 [==============================] - 0s 38ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 218/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 219/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.00

1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 282/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 283/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 284/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 285/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 286/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 287/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 288/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 289/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.00

...

1/1 [==============================] - 0s 44ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 702/1000
1/1 [==============================] - 0s 44ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 703/1000
1/1 [==============================] - 0s 45ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 704/1000
1/1 [==============================] - 0s 51ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 705/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 706/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 707/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 708/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 709/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.00

1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 772/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 773/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 774/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 775/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 776/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 777/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 778/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 779/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.00

1/1 [==============================] - 0s 38ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 842/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 843/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 844/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 845/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 846/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 847/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 848/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 849/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.00

1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 912/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 913/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 914/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 915/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 916/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 917/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 918/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 919/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.00

1/1 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 982/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 983/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 984/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 985/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 986/1000
1/1 [==============================] - 0s 60ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 987/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 988/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 989/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.00

In [166]:
#cmd tensorboard --logdir=. 을 통해서 log와 LSTM 레이어가 어떻게 학습되는지 관찰할 수 있다.

# 8. make predictions

In [100]:
actions[np.argmax(res[0])]

'good'

In [101]:
actions[np.argmax(y_test[0])]

'good'

# 9. save model

In [102]:
model.save('action.h5')

In [103]:
model.load_weights('action.h5')

# 10. Evaluation using confusion matrix and accuracy

In [104]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [105]:
yhat = model.predict(X_train)

In [106]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [107]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[ 0,  0],
        [ 0, 28]]], dtype=int64)

In [108]:
accuracy_score(ytrue,yhat)

1.0

# 11. Test in Real Time

In [117]:
colors = [(245,117,16)]
#def prob_viz(res,actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    return output_frame

In [119]:
res

array([1.], dtype=float32)

In [136]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
#1 Net detection variables

sequence = []
sentence = []
predictions=[]
threshold = 0.4

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret,frame = cap.read()

        #make detection
        image , results  = mediapipe_detection(frame,holistic)
        
        #draw_landmark
        draw_landmarks(image,results)

        #2. prediction logic
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:39]
        
        if len(sequence) == 39:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
        
        #3. vizs logic - 0.4보다 큰 수치를 가졌을 경우에 상태가 바뀌면 바뀐 상태로, 안바뀌면 안바뀐 상태로 
        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
                    
        if len(sentence)>5:
            sentence = sentence[-5:]
            
        #viz
        #image = prob_viz(res,actions,image,colors)
        cv2.rectangle(image, (0,0),(640,40), (245,117,16),-1)
        cv2.putText(image, ' '.join(sentence),(3,30), cv2.FONT_HERSHEY_SIMPLEX,1,
                   (255,255,255),2,cv2.LINE_AA)
        
        #show to screen
        cv2.imshow('OpenCV Feed',image)
        #breaking
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [1]:
keypoints

NameError: name 'keypoints' is not defined

In [124]:
np.unique(predictions[-15:])[0]==np.argmax(res)

True

In [125]:
res

array([1.], dtype=float32)

In [126]:
np.unique(predictions[-15:])[0] == np.argmax(res)

True

In [127]:
res = model.predict(np.expand_dims(sequence, axis=0))[0]

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [128]:
predictions

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [129]:
res

array([1.], dtype=float32)

In [131]:
sequence

[array([-0.07815345, -0.05702017, -1.15445685, -0.04500108, -0.17423768,
        -1.19701076, -0.01893606, -0.18481751, -1.19783056,  0.0069208 ,
        -0.19500549, -1.19977462,  0.08192717, -0.19426499, -1.06683087,
         0.00233323,  0.00558468, -1.07538199, -0.07391849,  0.03116065,
        -0.91888511,  0.42283672,  0.1743682 , -1.07232499, -0.11749722,
         0.34516796,  0.04999669]),
 array([ 0.0042059 , -0.00817569, -1.16180825,  0.03658029, -0.1280202 ,
        -1.19657457,  0.06402734, -0.13775116, -1.19745576,  0.09117997,
        -0.14708142, -1.19947493, -0.02684258, -0.10567005, -1.09600663,
        -0.04726592, -0.09854002, -1.09534037, -0.06766765, -0.09128644,
        -1.0956043 ,  0.15866727, -0.1378106 , -1.02549815, -0.04855936,
        -0.06771072, -0.50589001,  0.07496899,  0.07195515, -1.07051206,
        -0.00179024,  0.08511581, -0.92295712,  0.4904843 ,  0.25277641,
        -1.04632664, -0.08404482,  0.38847029,  0.00210656]),
 array([ 0.10450996,  0.11

In [630]:
len(sequence)

30